In [45]:
import pandas as pd
import numpy as np
import os
import csv

In [46]:
#Event='AscendEx'
Event='Upbit'
raw_path='../Dataset/'+Event+'/all_data_token/'
address_path='../Dataset/'+Event+'/dataset/'
#account_feature_output_path = '../Code_Results/Intermediary_Account_Feature_Results/'+Event+'_account_feature.csv'
input_path ='../Code_Results/Tx_Value_Results/Tx_ETH_Value.csv'

In [47]:
#Step1：筛选出之发生了ETH交易的洗钱账户（source+layering）
#方法2：all_erc20_address和all_normal_address的交集，再用all_normal_address减掉交集
# all_address=pd.read_csv(address_path+"all-address.csv")
# all_erc20_address=pd.read_csv(address_path+"all-erc20-address.csv")
all_ETH_tx=pd.read_csv('../Dataset/Normal_Tx/all-tx.csv')

# all_ETH_tx=pd.read_csv('../Dataset/Normal_Tx/all-tx-9968.csv')

In [48]:
all_ETH_tx=all_ETH_tx.loc[(all_ETH_tx['value'].astype(float)<=10**(6))]

In [49]:
all_ETH_tx

,Unnamed: 0,blockid,timestamp,txhash,type,fromac,toac,gaslimit,value,newcontract,code,isSuccessful
0,0,46466,1438923647,0x09abf3b250969ab5a4bc80533f407dfaca950bbfd833...,Call,0xa89ac93b23370472daac337e9afdf642543f3e57,0x0000000000000000000000000000000000000000,0.0,100.000000,NaN,NaN,OK
1,1,46554,1438925136,0x85e4a08b2033222d8900552d2da73fc04d7f9aef2f73...,Call,0xff68c2f39c4e7fe2f92c2dc2f8615b1bf070e0fa,0x0000000000000000000000000000000000000000,0.0,14.000000,NaN,NaN,OK
2,2,46857,1438930476,0xfb3118914c770009654a36904897ac12a1d69b8479b7...,Call,0xc8ebccc5f5689fa8659d83713341e5ad19349448,0x0000000000000000000000000000000000000000,0.0,14.000000,NaN,NaN,OK
3,3,46941,1438931909,0x38c4353975f860c577b8d43917a0ce9e26ffb4a10bef...,Call,0xc8ebccc5f5689fa8659d83713341e5ad19349448,0x0000000000000000000000000000000000000000,0.0,0.000010,NaN,NaN,OK
4,4,47441,1438940411,0x824b5c44d55e7a50f6f57600059dae63418586a011ee...,Call,0x3b7ce4d16316f594a3c5ebb7ccc4c022fc2858c6,0x0000000000000000000000000000000000000000,2300.0,1.000000,NaN,NaN,OK
...,...,...,...,...,...,...,...,...,...,...,...,...
776961,85,685374,1450008771,0xe2008946970ac9ad3510512c993fde1221822fcb7752...,Call,0xf8b483dba2c3b7176a3da549ad41a48bb3121069,0x1236eff935945d3644184f3584380a6be5aeb566,69000.0,1.363369,NaN,NaN,OK
776962,84,686150,1450020947,0x9b628516fc0377f25895cd4db5d463bf89344e432fd2...,Call,0x1236eff935945d3644184f3584380a6be5aeb566,0xc47aaa860008be6f65b58c6c6e02a84e666efe31,0.0,1.362319,NaN,NaN,OK
776963,86,688861,1450067282,0x477f8382dc675f3f872c059db0ca4de424d50469a7c5...,Call,0xf8b483dba2c3b7176a3da549ad41a48bb3121069,0x1236eff935945d3644184f3584380a6be5aeb566,69000.0,1.092335,NaN,NaN,OK
776964,85,689640,1450081818,0x1dc5ff6968651a1abd2652c4eb8846134a21ff3f582c...,Call,0x1236eff935945d3644184f3584380a6be5aeb566,0xc47aaa860008be6f65b58c6c6e02a84e666efe31,0.0,1.091285,NaN,NaN,OK


In [50]:
input_csv = pd.read_csv(input_path)

In [51]:
input_csv

,Tx_value,Num,Percent
0,0,NaN,NaN
1,1,NaN,NaN
2,2,NaN,NaN
3,3,NaN,NaN
4,4,NaN,NaN
5,5,NaN,NaN
6,6,NaN,NaN


In [52]:
#Step3：统计交易金额所在范围的数量
import warnings
warnings.filterwarnings("ignore")
tmp=len(all_ETH_tx.loc[(all_ETH_tx['value'].astype(float)<1)&(all_ETH_tx['value'].astype(float)>0)])
input_csv['Num'][0]=tmp                  
for i in range(1,len(input_csv['Tx_value'])):
    Tx_value_range=input_csv['Tx_value']
#     print(int(Tx_value_range[i]))
    min_value=10**(Tx_value_range[i].astype(float)-1)
    max_value=10**(Tx_value_range[i].astype(float))
#     print(min_value)
#     print(max_value)
    tmp=len(all_ETH_tx.loc[(all_ETH_tx['value'].astype(float)>=min_value)& (all_ETH_tx['value'].astype(float)<max_value)])
    #print(tmp)
    input_csv['Num'][i]=tmp
#     break
# tmp=len(all_ETH_tx.loc[(all_ETH_tx['value'].astype(float)>=5)])
# input_csv['Num'][6]=tmp   

In [53]:
input_csv['Percent']=input_csv['Num']/sum(input_csv['Num'])

In [54]:
input_csv

,Tx_value,Num,Percent
0,0,304027.0,0.391305
1,1,336854.0,0.433556
2,2,100917.0,0.129887
3,3,26086.0,0.033575
4,4,8743.0,0.011253
5,5,305.0,0.000393
6,6,25.0,0.000032


In [55]:
sum(input_csv['Percent'])

0.9999999999999999